In [1]:
"""
This Notebook gives an intro on how to handle BuildingMotif's sqlalchemy Session.
"""
import os

from rdflib import RDF, URIRef
from rdflib.compare import isomorphic
from rdflib.namespace import FOAF
import sqlite3 as lite

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Template, Library, Model

In [2]:
"""
Create a testdb file, init a buiding_motif, and create a sqlite connection (for db transparency)
"""
file = "./test.db"
if os.path.isfile(file):
    os.remove(file) 

building_motif = BuildingMOTIF(f"sqlite:///{file}")
building_motif.setup_tables()
building_motif.session

conn = lite.connect(file)
cur = conn.cursor()


In [3]:
"""
Create a Library. Note that while You can get
the library within the session, it is not writen
to the db until after the commit.
"""
lib = Library.create("my_library")
print(lib)
assert len(building_motif.table_connection.get_all_db_libraries()) == 1

cur.execute("SELECT * FROM library")
print(f"pre-commit library: {cur.fetchall()}")

building_motif.session.commit()

cur.execute("SELECT * FROM library")
print(f"post-commit library: {cur.fetchall()}")

Library(_id=1, _name='my_library', _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1058789d0>)
pre-commit library: []
post-commit library: [(1, 'my_library', 1)]


In [4]:
"""
Same thing with a template. 
"""
t = lib.create_template(name="my_template")
print(t)
assert lib.get_templates()[0] == t
assert len(building_motif.table_connection.get_all_db_templates()) == 1

cur.execute("SELECT * FROM template")
print(f"pre-commit template: {cur.fetchall()}")

building_motif.session.commit()

cur.execute("SELECT * FROM template")
print(f"post-commit template: {cur.fetchall()}")

Template(_id=1, _name='my_template', body=<Graph identifier=c1567106-ccea-4d56-b941-8911c7d89b5f (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x1058789d0>)
pre-commit template: []
post-commit template: [(1, 'my_template', 'c1567106-ccea-4d56-b941-8911c7d89b5f', '[]', 1)]


In [5]:
"""
Graphs abide by sessioning, too
"""
print(t.body)
t.body.add((URIRef("http://example.org/alex"), RDF.type, FOAF.Person))
body_id = building_motif.table_connection.get_db_template(t.id).body_id
assert isomorphic(building_motif.graph_connection.get_graph(body_id), t.body)

cur.execute("SELECT * FROM kb_625d302a74_type_statements")
print(f"pre-commit statements: {cur.fetchall()}")

building_motif.session.commit()

cur.execute("SELECT * FROM kb_625d302a74_type_statements")
print(f"post-commit statements: {cur.fetchall()}")



<c1567106-ccea-4d56-b941-8911c7d89b5f> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'SQLAlchemy'].
pre-commit statements: []
post-commit statements: [(1, 'http://example.org/alex', 'http://xmlns.com/foaf/0.1/Person', 'c1567106-ccea-4d56-b941-8911c7d89b5f', 0)]


In [6]:
"""
You don't have to commit after each create. After each
create, we flush the session, so that the created object
is avaiable for reference.
"""
lib = Library.create("your_library")
t = lib.create_template(name="your_template")
t.body.add((URIRef("http://example.org/alex"), RDF.type, FOAF.Person))

building_motif.session.commit()

In [7]:
"""
A db error could happen on a flush or a commit, and you
will have to rollback to continue using the session.
"""
try:
    Library.create("my_library")
except Exception as e:
    print(f"{e}\n")
    building_motif.session.rollback()
    
bad_lib = Library.create("a fine name")
bad_lib.name = "my_library"
try:
    building_motif.session.commit()
except Exception as e:
    print(f"{e}\n")
    building_motif.session.rollback()

(sqlite3.IntegrityError) UNIQUE constraint failed: library.name
[SQL: UPDATE library SET name=? WHERE library.id = ?]
[parameters: ('my_library', 3)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)



In [8]:
"""
Remember buiding_motif.session is a unwrapped or modified 
 Session, and using it comes with all the power
and complexity of . Be sure to read the docs!

https://docs.sqlalchemy.org/en/14/orm/session_api.html
"""

'\nRemember buiding_motif.session is a unwrapped or modified \n Session, and using it comes with all the power\nand complexity of . Be sure to read the docs!\n\nhttps://docs.sqlalchemy.org/en/14/orm/session_api.html\n'